# Opening a Vegan Restaurant in London

### Table of Contents

> <a href="#1.-Introduction">1. Introduction </a>
>
> <a href="#2.-Data-Acquisition-and-Preliminary-Analysis">2. Data Acquisition and Preliminary Analysis </a>
>
> <a href="#3.-Methodology">3. Methodology </a>
>
> <a href="#4.-Results">4. Results </a>
>
> <a href="#5.-Conclusion">5. Conclusion </a>
>
> <a href="#6.-References">6. References </a>

# 1. Introduction

Healthy food has become a trending topic in the past few years. People are paying more attention to what they eat, and how they eat. Several consumers are changing their habits from animal derived goods to plant-based goods, and the interest has increased significantly in the United Kingdom.

The Vegan Society [[1]](https://www.vegansociety.com/news/media/statistics#vegandietintheuk) has a collection of statistics with its sources that can present the growing potential of this market. Some interesting observations are extracted and presented here:
> The UK's purchase and consumption rates of vegan: milk, meat, butter/margarine, cheese, ready meals/food to go and seafood are the highest in Europe. [[2]](https://proveg.com/what-we-do/corporate-engagement/proveg-consumer-survey-report-download/)
>
> In 2018, the UK launched more vegan products than any nation. [[3]](https://www.mintel.com/press-centre/food-and-drink/veganuary-uk-overtakes-germany-as-worlds-leader-for-vegan-food-launches)
>
> Between November 2019 and November 2020, vegan food orders via Deliveroo shot up 115%. [[4]](https://www.casualdiningmagazine.co.uk/news/2020-10-29-vegan-orders-surge-more-than-100-in-12-months)
>
> Demand for meat-free food in the UK increased by 987% in 2017 and going vegan was predicted to be the biggest food trend in 2018. [[5]](https://plantbasednews.org/lifestyle/2017-ridiculous-987-increase-demand-meat-free-options/)

The Google Trends [[6]](https://trends.google.com/trends/explore?date=all&q=%2Fm%2F07_hy) on veganism puts the UK as the fourth region with more interest in the subject, and shows the increase in search of veganism consistently rising since 2013.

Considering these findings, a new business dedicated to vegan products seems a promising idea. There is possibility for profit together with a social and environmental impact on the society, values that are becoming more and more important for customers when deciding over what they are consuming. Also, there is still low competition in the segment, which buys time for the businees to stablish itself as pioneer and retain loyal customers.

This study will evaluate where to open a new vegan restaurant in London, capital of England and the UK. The goal is to retrieve where there is a lack of options of vegan/vegetarian restaurants, but also have available clientele (for example, regions with lots of restaurants or bars guarantee exposition).

Hopefully this report will not only reach people who intend to open a new profitable, sustainable and environmental-friendly business, but also share some light on the importance of evaluating our eating habits and at least reduce the consumption of animal derived products.

# 2. Data Acquisition and Preliminary Analysis

First of all, import the libraries that will be used throughout this analysis.

In [1]:
import requests
import folium
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

## 2.1 Supporting the business decision
Webscraping from finder.com [[7]](https://www.finder.com/uk/uk-diet-trends) retrieves some data on consumer's diets. This data creates the base to sustain the analysis.

In [2]:
url = 'https://www.finder.com/uk/uk-diet-trends'
table = pd.read_html(url)

In [3]:
print("The UK's current diet:")
t1 = table[0]
t1

The UK's current diet:


,Diet,Percent
0,Total vegetarian,6.55%
1,Total pescatarian,4.10%
2,Total vegan,2.10%
3,Total meat free,12.75%
4,Not following any diet,87.25%


In [4]:
print("The UK’s diet intentions by the end of 2020:")
t2 = table[1]
t2

The UK’s diet intentions by the end of 2020:


,Diet in 2020,Percent
0,Total vegetarian,11.35%
1,Total pescatarian,7.35%
2,Total vegan,4.15%
3,Total meat free,22.85%
4,No diet intentions,77.15%


In [5]:
print("Specific diet changes over 2020:")
t3 = table[2]
t3

Specific diet changes over 2020:


,Type of diet,Current population,People who intend to follow this diet,Population by end of 2020,% change
0,No-specific diet,45721918,-5292738,40429180,-11.60%
1,Vegan,1100470,1074269,2174739,97.60%
2,Vegetarian,3432419,2515361,5947780,73.30%
3,Pescatarian,2148537,1703109,3851646,79.30%


In [6]:
print("Which generations are ditching the meat?")
t4 = table[3]
t4

Which generations are ditching the meat?


,Generation,Percent ditching meat
0,Postmillennial,13.89%
1,Millennials,14.74%
2,Generation X,13.79%
3,Boomers,10.57%
4,Silent generation,8.86%


In [7]:
print("Are there gender differences when it comes to diets?")
t5 = table[4]
t5

Are there gender differences when it comes to diets?


,Intention,Female,Male
0,Planning to go meat free,8.84%,11.44%
1,Already meat free,14.57%,10.82%


In [8]:
print("Yearly cost of each diet per person:")
t6 = table[5]
t6

Yearly cost of each diet per person:


,Diet,Cost per year
0,Vegan Diet,"£2,073"
1,Normal Balanced Diet,"£2,002"
2,Pescatarian Diet,"£1,973"
3,Vegetarian Diet,"£1,545"


The above data shows that an increasing number of the UK population is changing their diet to a more plant-based one, and that new generations tend to be more adherent to this lifestyle. Also, the last table breaks the concept that going vegan is more expensive, which can be used to induce more people to try it.

The important thing as a business owner is to offer plenty of options so the transition to a vegan or vegetarian diet is not difficult for the people wanting to pursue that.

## 2.2 Selecting and Cleaning Data

We'll use the Foursquare API to retrieve the locations of the vegan/vegetarian restaurants in London. With these locations, we can determine the best place to open a new restaurant. First, set up the Foursquare credentials necessary to collect data.

In [9]:
# the credentials were removed due to privacy reasons
# if you want to run this notebook, register in Foursquare and replace your credentials here
CLIENT_ID = '' 
CLIENT_SECRET = '' 
VERSION = '20201031' 
LIMIT = 100

Let's create an initial map of London, where we'll visualise the restaurants.

In [10]:
city = 'London'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
london_map = folium.Map(location=[latitude, longitude], zoom_start=12)

We'll use the Foursquare API for venue's searching, considering the London location (latitude, longitude), and look for restaurants in a 22 km radius. This should cover the entire area of the Greater London. [[8]](https://en.wikipedia.org/wiki/Greater_London)

In [11]:
radius = 22000
search_query = 'Vegan'
url='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude,
    longitude, 
    VERSION,
    search_query,
    radius, 
    LIMIT)
results = requests.get(url).json()

In [12]:
# assign relevant part of JSON to vegan_venues
vegan_venues = results['response']['venues']

# tranform vegan_venues into a dataframe
vegan_df = pd.json_normalize(vegan_venues)
vegan_df.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood,location.crossStreet
0,59ada1f8febf314589c6f8d5,Vegan Yes,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",v-1604666807,False,64 Brick Ln,51.518734,-0.071483,"[{'label': 'display', 'lat': 51.518734, 'lng':...",4093,E1 6QL,GB,London,Greater London,United Kingdom,"[64 Brick Ln, London, Greater London, E1 6QL, ...",NaN,NaN,NaN
1,5848791af68d8d38cfcf6446,Vegan Burgers by Mooshies,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",v-1604666807,False,104 Brick Ln,51.519731,-0.071783,"[{'label': 'display', 'lat': 51.51973095696915...",4109,E1 6RL,GB,London,Greater London,United Kingdom,"[104 Brick Ln, London, Greater London, E1 6RL,...",NaN,NaN,NaN
2,5ad1edfe3ba7671c3ec4b6f8,Soho Vegan Market,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",v-1604666807,False,Rupert Street,51.512285,-0.133766,"[{'label': 'display', 'lat': 51.512285, 'lng':...",696,W1D 7PP,GB,London,Greater London,United Kingdom,"[Rupert Street, London, Greater London, W1D 7P...",NaN,NaN,NaN
3,5437b093498e11e3771b4a2c,Vegan Sweet Tooth,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1604666807,False,NaN,51.520467,-0.072473,"[{'label': 'display', 'lat': 51.52046677379518...",4092,NaN,GB,NaN,NaN,United Kingdom,[United Kingdom],NaN,NaN,NaN
4,5c2b803618d43b002ce1aae0,Korean BBQ & Vegan Restaurant,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",v-1604666807,False,5 Mile End Road,51.519983,-0.056017,"[{'label': 'display', 'lat': 51.519983, 'lng':...",5158,E1 4TP,GB,London,Greater London,United Kingdom,"[5 Mile End Road, London, Greater London, E1 4...",529127751,NaN,NaN


In [13]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in vegan_df.columns if col.startswith('location.')] + ['id']
vegan_df_filtered = vegan_df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
vegan_df_filtered['categories'] = vegan_df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
vegan_df_filtered.columns = [column.split('.')[-1] for column in vegan_df_filtered.columns]

vegan_df_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,crossStreet,id
0,Vegan Yes,Vegetarian / Vegan Restaurant,64 Brick Ln,51.518734,-0.071483,"[{'label': 'display', 'lat': 51.518734, 'lng':...",4093,E1 6QL,GB,London,Greater London,United Kingdom,"[64 Brick Ln, London, Greater London, E1 6QL, ...",NaN,NaN,59ada1f8febf314589c6f8d5
1,Vegan Burgers by Mooshies,Vegetarian / Vegan Restaurant,104 Brick Ln,51.519731,-0.071783,"[{'label': 'display', 'lat': 51.51973095696915...",4109,E1 6RL,GB,London,Greater London,United Kingdom,"[104 Brick Ln, London, Greater London, E1 6RL,...",NaN,NaN,5848791af68d8d38cfcf6446
2,Soho Vegan Market,Vegetarian / Vegan Restaurant,Rupert Street,51.512285,-0.133766,"[{'label': 'display', 'lat': 51.512285, 'lng':...",696,W1D 7PP,GB,London,Greater London,United Kingdom,"[Rupert Street, London, Greater London, W1D 7P...",NaN,NaN,5ad1edfe3ba7671c3ec4b6f8
3,Vegan Sweet Tooth,Food Truck,NaN,51.520467,-0.072473,"[{'label': 'display', 'lat': 51.52046677379518...",4092,NaN,GB,NaN,NaN,United Kingdom,[United Kingdom],NaN,NaN,5437b093498e11e3771b4a2c
4,Korean BBQ & Vegan Restaurant,Vegetarian / Vegan Restaurant,5 Mile End Road,51.519983,-0.056017,"[{'label': 'display', 'lat': 51.519983, 'lng':...",5158,E1 4TP,GB,London,Greater London,United Kingdom,"[5 Mile End Road, London, Greater London, E1 4...",NaN,NaN,5c2b803618d43b002ce1aae0


Let's check the categories column to see which types of places are being considered:

In [14]:
vegan_df_filtered['categories'].value_counts()

Vegetarian / Vegan Restaurant    36
Food Truck                        2
Bakery                            2
Tea Room                          1
Building                          1
Donut Shop                        1
Food Stand                        1
School                            1
Market                            1
Farmers Market                    1
Fried Chicken Joint               1
Café                              1
Organic Grocery                   1
Name: categories, dtype: int64

Let's take a closer a look into the places where the category doesn't fit into vegan/vegetarian restaurants.

In [15]:
vegan_df_filtered[vegan_df_filtered['categories']!='Vegetarian / Vegan Restaurant']

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,crossStreet,id
3,Vegan Sweet Tooth,Food Truck,NaN,51.520467,-0.072473,"[{'label': 'display', 'lat': 51.52046677379518...",4092,NaN,GB,NaN,NaN,United Kingdom,[United Kingdom],NaN,NaN,5437b093498e11e3771b4a2c
7,Vegan Crosstown Doughnuts,Donut Shop,5-6 Picton Pl,51.515354,-0.151135,"[{'label': 'display', 'lat': 51.51535387690779...",1856,W1U 1BW,GB,London,Greater London,United Kingdom,"[5-6 Picton Pl, London, Greater London, W1U 1B...",NaN,NaN,5aaf8519e2d4aa7dd23b9325
10,Astro Vegan,Food Stand,NaN,51.519875,-0.109297,"[{'label': 'display', 'lat': 51.519875, 'lng':...",1889,E C1N,GB,London,Greater London,United Kingdom,"[London, Greater London, E C1N, United Kingdom]",NaN,NaN,596364a48194fc32008e3118
19,London Vegetarian and Vegan School,School,217 Long Lane,51.498461,-0.083773,"[{'label': 'display', 'lat': 51.49846117038613...",3196,SE1 4PA,GB,London,Greater London,United Kingdom,"[217 Long Lane (Unit 6 Dun dee Ct.), London, G...",NaN,Unit 6 Dun dee Ct.,57b97dfc498e955a180e90e2
20,Vida Vegan Bakery,Bakery,139 Brick Lane,51.523909,-0.071618,"[{'label': 'display', 'lat': 51.523909, 'lng':...",4298,E1 6SB,GB,London,Greater London,United Kingdom,"[139 Brick Lane, London, Greater London, E1 6S...",NaN,NaN,5b214b9779f6c7002c292e6b
21,London Vegetarian and Vegan School,Building,NaN,51.498638,-0.083737,"[{'label': 'display', 'lat': 51.498638, 'lng':...",3192,NaN,GB,London,Greater London,United Kingdom,"[London, Greater London, United Kingdom]",NaN,NaN,57b9ad1a498ec3e4b51e80b1
25,Vegan Tea Room,Tea Room,56 Stoke Newington High St,51.552141,-0.076193,"[{'label': 'display', 'lat': 51.55214087733508...",6131,N16,GB,Hackney,Greater London,United Kingdom,"[56 Stoke Newington High St, Hackney, Greater ...",NaN,NaN,510e657ee4b03bf838e88434
26,Vegan Jesus,Café,The yard,51.469730,-0.066555,"[{'label': 'display', 'lat': 51.46973, 'lng': ...",5953,SE15 4UJ,GB,London,Greater London,United Kingdom,"[The yard, London, Greater London, SE15 4UJ, U...",NaN,NaN,59bced6f7dc9e174b5311266
31,Vegan Sweet Tooth London,Food Truck,60 Chatsworth Rd,51.549477,-0.039892,"[{'label': 'display', 'lat': 51.54947710719424...",7678,E5 0LH,GB,London,Greater London,United Kingdom,"[60 Chatsworth Rd, London, Greater London, E5 ...",NaN,NaN,5435cc13498e3f9f848d52ae
33,Brixton Vegan Market,Farmers Market,NaN,51.463398,-0.114214,"[{'label': 'display', 'lat': 51.463398, 'lng':...",4977,NaN,GB,NaN,NaN,United Kingdom,[United Kingdom],NaN,NaN,5a08600628122f18d5b637b9


We can see that, although related to vegan culture and products, these places really don't fall into our main category, Vegan / Vegetarian Restaurants. These values are returned by Foursquare search because of the relation through the word "vegan".
Considering this observation, it's fair to keep the food truck and food stand venues, since they offer the same as common restaurants. So, let's clean the dataframe to remove the unwanted values.

In [16]:
vegan = vegan_df_filtered
vegan.drop(vegan[(vegan['categories']!='Vegetarian / Vegan Restaurant') & 
                 (vegan['categories']!='Food Truck') & 
                 (vegan['categories']!='Food Stand')
                ].index,
           inplace=True
          )
vegan.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,crossStreet,id
0,Vegan Yes,Vegetarian / Vegan Restaurant,64 Brick Ln,51.518734,-0.071483,"[{'label': 'display', 'lat': 51.518734, 'lng':...",4093,E1 6QL,GB,London,Greater London,United Kingdom,"[64 Brick Ln, London, Greater London, E1 6QL, ...",NaN,NaN,59ada1f8febf314589c6f8d5
1,Vegan Burgers by Mooshies,Vegetarian / Vegan Restaurant,104 Brick Ln,51.519731,-0.071783,"[{'label': 'display', 'lat': 51.51973095696915...",4109,E1 6RL,GB,London,Greater London,United Kingdom,"[104 Brick Ln, London, Greater London, E1 6RL,...",NaN,NaN,5848791af68d8d38cfcf6446
2,Soho Vegan Market,Vegetarian / Vegan Restaurant,Rupert Street,51.512285,-0.133766,"[{'label': 'display', 'lat': 51.512285, 'lng':...",696,W1D 7PP,GB,London,Greater London,United Kingdom,"[Rupert Street, London, Greater London, W1D 7P...",NaN,NaN,5ad1edfe3ba7671c3ec4b6f8
3,Vegan Sweet Tooth,Food Truck,NaN,51.520467,-0.072473,"[{'label': 'display', 'lat': 51.52046677379518...",4092,NaN,GB,NaN,NaN,United Kingdom,[United Kingdom],NaN,NaN,5437b093498e11e3771b4a2c
4,Korean BBQ & Vegan Restaurant,Vegetarian / Vegan Restaurant,5 Mile End Road,51.519983,-0.056017,"[{'label': 'display', 'lat': 51.519983, 'lng':...",5158,E1 4TP,GB,London,Greater London,United Kingdom,"[5 Mile End Road, London, Greater London, E1 4...",NaN,NaN,5c2b803618d43b002ce1aae0


Now we can plot the results to check where the places that offer prepared vegan / vegetarian food are located.

In [17]:
# add the vegan restaurants as green circle markers
for lat, lng, label in zip(vegan.lat, vegan.lng, vegan.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='#90EE90',
        fill_opacity=0.5
    ).add_to(london_map)

# display map
london_map

If you are just reading this notebook on GitHub, the Folium maps are not rendered. So, I decided to put a figure of the rendered map as well. It's static, so if you want to zoom in or out, you'll have to download and run the notebook wherever is convenient for you.
![Vegan Restaurants in London](https://raw.githubusercontent.com/rcdpereira/Coursera_Capstone/main/Vegan_Restaurants_London_Map.png)

# 3. Methodology

In this section, we're going to make an exploratory analysis of the London districts, in order to find where can be the best place to open a vegetarian / vegan restaurant. We want to search all London districts, find the ones that have the highest number of venues (busy districts), and check if the district already have a vegetarian / vegan restaurant. If it is a busy district and doesn't have a plant-based option for eating out, this is a district that is worth investing in.

## 3.1 Retrieving Location of London Districts

Let's retrieve a list with the districts in London.

In [18]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_areas_of_London'
london_list = pd.read_html(wiki_url)
london = london_list[1]
london.head()

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


And make some cleaning and adjustments.

In [19]:
london.drop(columns=['Post town', 'Postcode\xa0district', 'Dial\xa0code', 'OS grid ref'], inplace=True)

In [20]:
london.rename(columns={'Location':'District','London\xa0borough':'Borough'}, inplace=True)

Some of the disctricts are written with observations in parenthesis, which can compromise the geolocator reading. This is originally obtained by running the geolocator and finding the places where no latitude/longitude values were retrieved. We'll skip this step to avoid running the geolocator twice.

In [21]:
london.at[25,'District']= 'Barnet'
london.at[44,'District']= 'Bexley'
london.at[45,'District']= 'Bexleyheath'
london.at[66,'District']= 'Bromley'
london.at[302,'District']= 'Marylebone'
london.at[454,'District']= 'Sydenham'
london.at[517,'District']= 'Widmore'

Let's check for duplicates in the name of the districts, since this can lead to putting the same location into different places.

In [22]:
duplicate = london[london.duplicated(['District'], keep = False)]
duplicate

,District,Borough
37,Belmont,Harrow[22]
38,Belmont,Sutton[22]
65,Bromley,Bromley[35]
66,Bromley,Tower Hamlets[36]
100,Church End,Brent
101,Church End,Barnet
111,Coombe,Croydon
112,Coombe,Kingston upon Thames
196,Grove Park,Hounslow
197,Grove Park,Lewisham


Since there are few, we'll manually change the names of the duplicated districts to include the name of the borough. This will allow us to differentiate places and apply the district name to find the correct geolocation of each place.

In [23]:
london.at[37, 'District'] = 'Belmont, Harrow'
london.at[38, 'District'] = 'Belmont, Sutton'
london.at[65, 'District'] = 'Bromley, Bromley'
london.at[66, 'District'] = 'Bromley, Tower Hamlets'
london.at[100, 'District'] = 'Church End, Brent'
london.at[101, 'District'] = 'Church End, Barnet'
london.at[111, 'District'] = 'Coombe, Croydon'
london.at[112, 'District'] = 'Coombe, Kingston upon Thames'
london.at[196, 'District'] = 'Grove Park, Hounslow'
london.at[197, 'District'] = 'Grove Park, Lewisham'
london.at[230, 'District'] = 'Hayes, Bromley'
london.at[231, 'District'] = 'Hayes, Hillingdon'
london.at[367, 'District'] = 'Plaistow, Newham'
london.at[368, 'District'] = 'Plaistow, Bromley'

In [24]:
london[london.duplicated(['District'], keep = False)]

,District,Borough


Now that there are no duplicated values in the District column, let's get the location of the districts.

In [25]:
geolocator = Nominatim(user_agent="london_agent")
district_lat = []
district_lon = []
for n in range (0, len(london)):
    district = '{}, London, UK'.format(london['District'][n])
    loc = geolocator.geocode(district)
    if loc != None:
        district_lat.append(loc.latitude)
        district_lon.append(loc.longitude)
    else:
        district_lat.append('Nan')
        district_lon.append('Nan')

And append the locations to the london dataframe.

In [26]:
london['Latitude'] = district_lat
london['Longitude'] = district_lon
london.head()

,District,Borough,Latitude,Longitude
0,Abbey Wood,"Bexley, Greenwich [7]",51.4876,0.11405
1,Acton,"Ealing, Hammersmith and Fulham[8]",51.5081,-0.273261
2,Addington,Croydon[8],51.3586,-0.0316347
3,Addiscombe,Croydon[8],51.3797,-0.0742821
4,Albany Park,Bexley,51.4354,0.125965


Check if a district's location wasn't able to be retrieved.

In [27]:
london.loc[london[london['Latitude'] == 'Nan'].index]

,District,Borough,Latitude,Longitude
5,Aldborough Hatch,Redbridge[9],Nan,Nan
301,Marylebone (also St Marylebone),Westminster,Nan,Nan
410,Somerstown,Camden,Nan,Nan
453,"Sydenham (also Lower Sydenham, Upper Sydenham)","Lewisham, Bromley",Nan,Nan
516,Widmore (also Widmore Green),Bromley,Nan,Nan


Since there were only five districts with these results, instead of correcting let's just drop these values.

In [28]:
london.drop(london.loc[london['District'] == 'Aldborough Hatch'].index, inplace=True)
london.drop(london.loc[london['District'] == 'Marylebone (also St Marylebone)'].index, inplace=True)
london.drop(london.loc[london['District'] == 'Somerstown'].index, inplace=True)
london.drop(london.loc[london['District'] == 'Sydenham (also Lower Sydenham, Upper Sydenham)'].index, inplace=True)
london.drop(london.loc[london['District'] == 'Widmore (also Widmore Green)'].index, inplace=True)

## 3.2 Exploring Venues in London Districts

Now let's explore our districts.

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
london_venues = getNearbyVenues(names=london['District'],
                                   latitudes=london['Latitude'],
                                   longitudes=london['Longitude']
                                  )

In [32]:
print(london_venues.shape)
london_venues.head()

(12390, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abbey Wood,51.487621,0.114050,Co-op Food,51.487650,0.113490,Grocery Store
1,Abbey Wood,51.487621,0.114050,Neem Tree Pharmacy in Abbeywood,51.487543,0.113361,Pharmacy
2,Abbey Wood,51.487621,0.114050,Bostal Gardens,51.486670,0.110462,Playground
3,Abbey Wood,51.487621,0.114050,Abbey Wood Caravan Club,51.485502,0.120014,Campground
4,Acton,51.508140,-0.273261,London Star Hotel,51.509624,-0.272456,Hotel


How many unique categories of venues are retrieved?

In [34]:
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 413 uniques categories.


Analysing each district:

In [35]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add district column back to dataframe
london_onehot['District'] = london_venues['District'] 

# move district column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,District,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,...,Windmill,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Yoshoku Restaurant,Zoo,Zoo Exhibit
0,Abbey Wood,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Abbey Wood,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Abbey Wood,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Abbey Wood,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Acton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Let's check which districts have more venues:

In [36]:
london_top_20 = pd.DataFrame(london_onehot['District'].value_counts().head(20))
london_top_20

,District
"Coombe, Kingston upon Thames",100
St Giles,100
Canary Wharf,100
Shepherd's Bush,100
Soho,100
Aldwych,100
Shoreditch,100
Aldgate,100
Brompton,100
Clerkenwell,100


And let's rearrange the dataframe with more precise information.

In [37]:
london_top_20.reset_index(drop=False, inplace=True)

In [38]:
london_top_20.rename(columns={'index':'District', 'District': '# of Venues'}, inplace=True)
london_top_20

,District,# of Venues
0,"Coombe, Kingston upon Thames",100
1,St Giles,100
2,Canary Wharf,100
3,Shepherd's Bush,100
4,Soho,100
5,Aldwych,100
6,Shoreditch,100
7,Aldgate,100
8,Brompton,100
9,Clerkenwell,100


Now we need to find which districts already have a vegetarian / vegan restaurant. We wanto to avoid direct competition, since we want these places to thrive.

Grouping rows by district and taking the mean of the frequency of occurrence of each category gives us:

In [39]:
london_grouped = london_onehot.groupby('District').mean().reset_index()
london_grouped

,District,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,...,Windmill,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Yoshoku Restaurant,Zoo,Zoo Exhibit
0,Abbey Wood,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Acton,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Addington,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Addiscombe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Albany Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,Woolwich,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
518,Worcester Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
519,Wormwood Scrubs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
520,Yeading,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 3.3. FInding the Best Districts for Business

In this section, we are going to mix together our previously gathered information. A simple comparison will be enough to give us want we are looking for.

From the london_onehot dataframe, we can check how many vegan restaurants were returned when exploring London.

In [40]:
london_onehot['Vegetarian / Vegan Restaurant'].value_counts()

0    12353
1       37
Name: Vegetarian / Vegan Restaurant, dtype: int64

This is consistent enough with our vegan dataframe, which only contains vegetarian / vegan restaurants:

In [41]:
vegan.shape

(39, 16)

First, we want to see which districts have Vegetarian / Vegan Restaurants.

In [42]:
london_vegan = london_grouped[['District', 'Vegetarian / Vegan Restaurant']]
london_vegan.head()

,District,Vegetarian / Vegan Restaurant
0,Abbey Wood,0.0
1,Acton,0.0
2,Addington,0.0
3,Addiscombe,0.0
4,Albany Park,0.0


In [43]:
london_vegan.loc[london_vegan['Vegetarian / Vegan Restaurant'] > 0.01]

,District,Vegetarian / Vegan Restaurant
11,Archway,0.028571
42,Bethnal Green,0.016129
74,Cambridge Heath,0.023256
75,Camden Town,0.022989
85,Chalk Farm,0.012346
96,Chingford,0.066667
123,Crouch End,0.015625
130,Dalston,0.010753
133,Deptford,0.019608
135,Dollis Hill,0.125000


And ironically put the districts that doesn't have a vegan restaurant in the dataframe london_meat.

In [44]:
london_meat= london_vegan[london_vegan['Vegetarian / Vegan Restaurant'] < 0.01]
print(london_meat.shape)
london_meat

(488, 2)


,District,Vegetarian / Vegan Restaurant
0,Abbey Wood,0.0
1,Acton,0.0
2,Addington,0.0
3,Addiscombe,0.0
4,Albany Park,0.0
...,...,...
517,Woolwich,0.0
518,Worcester Park,0.0
519,Wormwood Scrubs,0.0
520,Yeading,0.0


Yep, from 522 initial districts, 488 doesn't have a vegetarian / vegan restaurant. That's a vast sea for business.
But since we don't want to open a restaurant that is far from movement of people, let's check which of these 488 districts are intersecting with our list of top 20 districts with most venues.

In [45]:
business_districts = pd.merge(london_meat, london_top_20, how='inner', on=['District']) 
business_districts

,District,Vegetarian / Vegan Restaurant,# of Venues
0,Aldwych,0.0,100
1,Brompton,0.0,100
2,Canary Wharf,0.0,100
3,Chinatown,0.0,100
4,Clerkenwell,0.0,100
5,Covent Garden,0.0,100
6,Ealing,0.0,96
7,Fitzrovia,0.0,96
8,Nag's Head,0.0,100
9,Richmond,0.0,100


# 4. Results

From our initial 527 districts of London that were evaluated, we were able to retrieve 15 where one can find lots of venues (and lots of people), but do not have an option for plant-based food.

And just for better visualisation, let's plot these districts in a map to see where they are, together with the existing vegetarian / vegan restaurants.

In [46]:
final_districts = pd.merge(business_districts, london, how='inner', on=['District'])
final_districts.drop(columns=['Borough'], inplace=True)
final_districts

,District,Vegetarian / Vegan Restaurant,# of Venues,Latitude,Longitude
0,Aldwych,0.0,100,51.5116,-0.119024
1,Brompton,0.0,100,51.4918,-0.178326
2,Canary Wharf,0.0,100,51.5049,-0.0190006
3,Chinatown,0.0,100,51.5117,-0.130496
4,Clerkenwell,0.0,100,51.5237,-0.105555
5,Covent Garden,0.0,100,51.5129,-0.122544
6,Ealing,0.0,96,51.5127,-0.305195
7,Fitzrovia,0.0,96,51.5188,-0.141002
8,Nag's Head,0.0,100,51.513,-0.12362
9,Richmond,0.0,100,51.4614,-0.303277


In [47]:
for lat, lng, label in zip(final_districts.Latitude, final_districts.Longitude, final_districts.District):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='#90EE90',
        fill_opacity=0.2
    ).add_to(london_map)

# display map
london_map

We can see that Covent Garden, Brompton and South Kensington for example are three of the districts that could be the first on the list to be explored. This opens the door to more detail investigation each of these districts considering several other things when deciding to open a restaurant (or any business), like rent, availability of property, safety, etc.

# 5. Conclusion

This simple analysis reveals the power of data analysis to help businesses achieve their goals. We started looking at the whole city of London, and finished looking to 15 districts that seem more promising, and even getting the first three on where to start using visualisation tools.

There is room for a more detailed analysis, which requires more work, and we can even get to the best streets where to open our restaurant. But I need to keep working, so this will be for another time.

# 6. References


1. https://www.vegansociety.com/news/media/statistics#vegandietintheuk

2. https://proveg.com/what-we-do/corporate-engagement/proveg-consumer-survey-report-download/

3. https://www.mintel.com/press-centre/food-and-drink/veganuary-uk-overtakes-germany-as-worlds-leader-for-vegan-food-launches

4. https://www.casualdiningmagazine.co.uk/news/2020-10-29-vegan-orders-surge-more-than-100-in-12-months

5. https://plantbasednews.org/lifestyle/2017-ridiculous-987-increase-demand-meat-free-options/

6. https://trends.google.com/trends/explore?date=all&q=%2Fm%2F07_hy

7. https://www.finder.com/uk/uk-diet-trends

8. https://en.wikipedia.org/wiki/Greater_London

9. https://plantproteins.co/vegan-plant-based-diet-statistics/

10. https://foodrevolution.org/blog/vegan-statistics-global/

11. https://healthcareers.co/vegan-statistics/

12. https://www.forbes.com/sites/janetforgrieve/2018/11/02/picturing-a-kindler-gentler-world-vegan-month/?sh=34e448e22f2b
